In [125]:
import requests
import re
url = 'https://pharmacy.buffalo.edu/faculty-staff.html'
i = 0

In [182]:
directory = requests.get('https://pharmacy.buffalo.edu/faculty-staff.html')
txt = directory.text
directory_text = txt.replace('>','>\n')
url_pattern = r'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile\.html\?ubit=\w+\b'
url_matches = re.findall(url_pattern,directory_text)

person_url = url_matches[1]
person_page = requests.get(person_url)
person_text = person_page.text
person_text = person_text.replace('>','>\n')
    #print(person_text)

split1 = person_text.split('<span id="lblAcademicOfficeAddressMobile">')[1]
no_campus = re.sub(r'\n\*{2}\s\w{5}\sCampus\s\*{2}', '', split1.replace('<br/>', ''))

print(no_campus)


453 Pharmacy Building
Buffalo, NY 14214-8033 
Phone: (716) 645-6112 
<a href="mailto:adolf@buffalo.edu">
adolf@buffalo.edu</a>

<div class="facultyprofilemapicon">
<a href="http://maps.google.com/maps?q=453+Pharmacy+Building+Buffalo+NY+14214-8033" target="_blank">
View map</a>
</div>
</span>
</div>

<div class="facultyprofile-expandablecontent">
</div>

</div>
</div>
</div>
</div>
<div class="facultyprofilebreaker">
<hr class="facultyprofilehorizonalbreaker"/>
</div>
<div class="facultyprofilecontents">
<div id="facultyprofiletabs" class="facultyprofiletabs">
<div class="facultyprofilecontentnavcontainer">
<div class="facultyprofilecontentnav">
<span class="facultyprofilecontentnav-tab-arrow">
</span>
 <a href="#publications" id="publications">
Publications</a>
 <a href="#latestnews" id="latestnews">
Latest News</a>
</div>
</div>
<div class="facultyprofilecontentbodies">
<div class="facultyprofilecontentbody facultyprofileoverview">
<noscript>
<a name="overview">
</a>
<h3>
Overview:</

In [169]:
def fetch_fix_names(raw_data):
    
    fetched_name = re.findall(r'\s?\b\w{,26}\s\w?\W?\s?\w{,26}\W{,2}\bspan\b\W\n', raw_data)
    
    if len(fetched_name) > 0:
        raw_name = fetched_name[0].replace('\n', '')
        raw_name = raw_name.replace('</span>', '')
    
        name = re.sub(r'\W\W\bspan\b\W', '', raw_name)
        
        return name
    
    else:
        return 'No Name'

In [170]:
split2 = person_text.split('<div class="facultyprofilemapicon">')[0]
split3 = split2.split('<span id="lblFacultyName">')[1]
#print(split3)
#print(re.findall(r'\s?\b[A-Z][a-z]{,26}\s[A-Z]?[(.)]\s?[A-Z][a-z]{,26}[(<)]\Wspan[(>)]\n', split3))

In [172]:
def fetch_fix_emails(raw_data):
    
    fetched_email = re.findall(r'(\Wa\s\bhref\b\=\"\bmailto\b\:[a-zA-Z0-9]{,25}\@\w{,26}\.\w{,10}\"\>\n)|(\w{,26}\@\w{,26}\.\w{,10}\<\/a\>\n)',
                               raw_data)
    
    if len(fetched_email) > 0:
        raw_data = fetched_email[0][0]
        raw_data = raw_data.replace('<a href="mailto:', '')
        email = raw_data.replace('">\n', '')
        
        return email
        
    else:
        return 'No email'

In [173]:
def fetch_fix_phones(raw_data):
    
    fetched_phone = re.findall(r'\b\w{4}e\W{,3}?\d{3}\W{,2}?\d{3}\W{,2}?\d{4}', raw_data)
        
    if len(fetched_phone) > 0:
        phone = fetched_phone[0]
        remove_label = phone.replace('Phone: ', '')
            
        s = remove_label.translate({ord(i): None for i in '() -.'})
        #phone_nums.append(remove_spaces)
        return ('(' + s[:3] + ')' + ' ' + s[3:6] + '-' + s[6:10])
            
    else:
        return 'No Phone'

In [190]:
def fetch_fix_address(raw_data):
    
    raw_data = re.sub(r'\n\*{2}\s\w{5}\sCampus\s\*{2}', '', raw_data)
    fetched_address = re.findall(r'\n.*?\n\b\w{7}\b\W{2}\b\w{2}\b\s?\d{,5}\-?\d{,4}', raw_data)
    
    if len(fetched_address) > 0:
        return fetched_address[0]
    
    else:
        return '\nNo Address'

In [191]:
def get_contact_block(url):
    contact_block = []
    
    directory = (requests.get(url)).text
    url_matches = re.findall(r'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile\.html\?ubit=\w+\b', 
                             directory.replace('>','>\n'))
    
    for person_url in url_matches:
        i = url_matches.index(person_url)
        person_page = requests.get(person_url)
        person_text = person_page.text
        person_text = person_text.replace('>','>\n')
    
        phone_address = person_text.split('<span id="lblAcademicOfficeAddressMobile">')[1]
        phone_address2 = phone_address.split('<div class="facultyprofilemapicon">')[0]
        phone_address_fix = phone_address2.replace('<br/>', '')
        phone = fetch_fix_phones(phone_address_fix)
        address = fetch_fix_address(phone_address_fix)
        
        
        name_email = person_text.split('<div class="facultyprofilemapicon">')[0]
        name_email2 = name_email.split('<span id="lblFacultyName">')[1]
        name = fetch_fix_names(name_email2)
        email = fetch_fix_emails(name_email2)

        
        contact_block.append(name + address + '\n' + phone + '\n' + email + '\n')
        
    return contact_block

contacts = get_contact_block(url)

In [194]:
for c in contacts:
    print(c)

Kimberly Adams
117 Pharmacy Building
Buffalo, NY 
(716) 645-4833
kadams4@buffalo.edu

Maureen A. Adolf
453 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-6112
adolf@buffalo.edu

Nicole P. Albanese
209 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-3915
npaolini@buffalo.edu

Kristopher M. Attwood
No Address
No Phone
attwood3@buffalo.edu

Joseph P. Balthasar
452 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-4807
jb@buffalo.edu

Contact Information<span>
359 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-4836
svb@buffalo.edu

Brenda S. Basile
345 Abbott Hall
Buffalo, NY 14214
No Phone
bsbasile@buffalo.edu

Edward M. Bednarczyk
222 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-4805
eb@buffalo.edu

Cindy J. Bednasz
B2-135B NYS Center of Excellence
Buffalo, NY 
(716) 881-8296
amabile@buffalo.edu

Natasha Benning
126 Cooke Hall
Buffalo, NY 
(716) 645-2823
benning2@buffalo.edu

Michele R. Bewley
445 Pharmacy Building
Buffalo, NY 14214-8033
(716) 645-2138
mrbewley@buffalo.